In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pyecharts import options as opts
from pyecharts.charts import Bar
from pyecharts.globals import ThemeType
from pyecharts.charts import Line
from pyecharts.charts import Pie


# data loading

In [2]:
df = pd.read_excel('Data_Engineer_Avon.xlsx', sheet_name='Data')
df2 = pd.read_excel('Data_Engineer_Avon.xlsx', sheet_name='Data2')

In [3]:
df

,Week,Mobile Indicator Name,Platform Type Name,Super Region,Country Name,Transfer Type,Receive Country,Net Sent Amount GBP,Net Orders
0,2019-W48,Desktop,Desktop,APAC,Australia,Mobile Money,Argentina,150829.00,5
1,2019-W48,Mobile,Mobile Web,APAC,Australia,Mobile Money,Argentina,27309.00,1
2,2019-W48,Mobile,Mobile Web,APAC,Australia,Mobile Money,Kenya,3290404.00,79
3,2019-W48,Mobile,Mobile App,APAC,Australia,Mobile Money,Kenya,1148144.00,30
4,2019-W48,Desktop,Desktop,APAC,Australia,Mobile Money,Kenya,12329935.00,282
...,...,...,...,...,...,...,...,...,...
73683,2018-W45,Mobile,Mobile App,NaN,US,Next Day Bank Transfer,Vietnam,296.10,1
73684,2018-W45,Desktop,Desktop,NaN,US,Next Day Bank Transfer,Zambia,1275.07,1
73685,2018-W45,Desktop,Desktop,NaN,US,Next Day Bank Transfer,Zimbabwe,766.57,2
73686,2018-W45,Desktop,Desktop,NaN,US,Unknown,Nigeria,8273.51,190


# Data Understanding 

data understanding is very important , we are going to check the following areas:
1. how many rows and columns are in the dataset
2. null values report - which columns have null values and can they be imputed during cleaning and feature engineering 
3. column sanity checks -  What are the data types of our column 


#### Data 1 

In [4]:
print("shape of dataframe", df.shape)
# obtaining the number of rows
print("number of rows : ", df.shape[0]) 
# obtaining the number of columns
print("number of columns : ", df.shape[1])

shape of dataframe (73688, 9)
number of rows :  73688
number of columns :  9


In [5]:
# how many missing values exist or better still what is the % of missing values in the dataset?
def percent_missing(df):

    # Calculate total number of cells in dataframe
    totalCells = np.product(df.shape)

    # Count number of missing values per column
    missingCount = df.isnull().sum()

    # Calculate total number of missing values
    totalMissing = missingCount.sum()

    # Calculate percentage of missing values
    print("The  dataset contains", round(((totalMissing/totalCells) * 100), 2), "%", "missing values.")

percent_missing(df)

The  dataset contains 3.06 % missing values.


which of the column contribute to the 3.06% missing?

In [6]:
df.isna().sum()

Week                         0
Mobile Indicator Name        0
Platform Type Name           0
Super Region             20265
Country Name                 0
Transfer Type                0
Receive Country              0
Net Sent Amount GBP          0
Net Orders                   0
dtype: int64

Observation : We can establish that Super region is the only column with Null values. During Data Cleaning we can explore further and try and fill the data if possible

What data types do we have in dataset 1?


In [7]:
for col in df.columns:
    print(col, ':', df[col].dtype)

Week : object
Mobile Indicator Name : object
Platform Type Name : object
Super Region : object
Country Name : object
Transfer Type : object
Receive Country : object
Net Sent Amount GBP : float64
Net Orders : int64


Observation : We have 2 numerical column and 7 categorical columns 

#### Data 2

In [8]:
print("shape of data2", df2.shape)
# obtaining the number of rows
print("number of rows : ", df2.shape[0]) 
# obtaining the number of columns
print("number of columns : ", df2.shape[1])

shape of data2 (168, 6)
number of rows :  168
number of columns :  6


In [9]:
percent_missing(df2)

The  dataset contains 2.38 % missing values.


Which columns are contributing to the 2.38% of null values?

In [10]:
df2.isna().sum()

Rpt Week                  0
Super Region             24
Country Name              0
Mobile Indicator Name     0
Platform Type Name        0
Visits                    0
dtype: int64

##### Observation : same as data 1 Super region is the only contributor

# EDA

In [29]:
def create_stacked_bar(df, column1, column2):

    bar = Bar()
    bar.add_xaxis(df[column1].unique().tolist())
    for platform in df[column2].unique().tolist():
        data = df[df[column2] == platform]['growth'].tolist()
        bar.add_yaxis(platform, data, stack='stack')
    bar.set_global_opts(xaxis_opts={"name":column1}, yaxis_opts={"name":"Growth(%)"})
    return bar.render_notebook()

def plot_line(df1,df2,x_name,y_name):
    line = Line()
    line.add_xaxis(df1.index.tolist())
    line.add_yaxis('2018', df1.values.tolist(), is_smooth=True)
    line.add_yaxis('2019', df2.values.tolist(), is_smooth=True)
    line.set_global_opts(xaxis_opts={"name":x_name}, yaxis_opts={"name":y_name})
    return line.render_notebook()

## Univariate Analysis 
Uni means one, In this case we are going to analyse every column individually 

In [11]:
# this code loops through every column to check the type, if any unique values, value count and some statistics for numerical columns 
def univarite(df):
    for column in df.columns:
        if df[column].dtype == object:
            print("Column: ", column)
            print("Unique Values: ", df[column].nunique())
            print("Value Counts: \n", df[column].value_counts())
            print("\n")
        else:
            print("Column: ", column)
            print("Mean: ", df[column].mean())
            print("Median: ", df[column].median())
            print("Minimum: ", df[column].min())
            print("Maximum: ", df[column].max())
            print("Standard Deviation: ", df[column].std())
            print("\n")
        


In [12]:
univarite(df)

Column:  Week
Unique Values:  10
Value Counts: 
 2019-W46    10154
2019-W45    10147
2019-W48     9919
2019-W47     9769
2018-W46     8536
2018-W45     8404
2018-W48     8370
2018-W47     8369
2018~w48       11
2019~W45        9
Name: Week, dtype: int64


Column:  Mobile Indicator Name
Unique Values:  5
Value Counts: 
 Mobile     38610
Desktop    35048
mobi          19
M@#3le        10
mibile         1
Name: Mobile Indicator Name, dtype: int64


Column:  Platform Type Name
Unique Values:  3
Value Counts: 
 Desktop       35048
Mobile Web    19852
Mobile App    18788
Name: Platform Type Name, dtype: int64


Column:  Super Region
Unique Values:  3
Value Counts: 
 APAC     23805
EMEA     23477
LATAM     6141
Name: Super Region, dtype: int64


Column:  Country Name
Unique Values:  7
Value Counts: 
 US                20265
United Kingdom    14873
South Korea        9878
Norway             8604
Australia          7225
Hong Kong          6702
Brazil             6141
Name: Country Name, dtype: 

Observation:
- Column : Week.
    - We can oberve that most transaction happened in week 46 for both 2018 and 2019 
    - We will also need some cleaning in this column, noticed ~ while others had - . we will need to use a standard -

- Column : Mobile Indicator Name.
    - Highest indicator is Mobile 
    - we will also need cleaning noticed some misspelling 'Mobi','m@#3le' and 'mibile' all will be converted to Mobile

- Column : Platform Type Name.
    - Seem clean 
    - we have 3 unique values, Highest count under Desktop type
    - point to note we just observe Mobile Indicator being high while desktop is high under platform?

- Column : Super Region.
    - 3 Main Supper Region 
    - most data is from APAC
- Column : Country Name (Sending Country).
    - 7 Main countries sending 
    - most are sent from USA

- Column : Tranfer Type
    - most transfer are 'Same day cash pick up'
    - Noticed some 'Unkown' Tranfer type , Are they Fraud transactions?, What countries are they from ?
    - cleaning is needed ~ to - 

- Column: Receive Country.
    - We will need to convert all names to lowercase

- Column: Net sent Amount GBP
    - Mean:  17743.3696377877
    - Median:  886.0009500000001
    - Minimum:  -347740.0 (from cancalations)
    - Maximum:  22886137.0
    - Standard Deviation:  290851.6228661975

- Column : Net Orders
    - Mean:  47.95460590598198
    - Median:  2.0
    - Minimum:  -50
    - Maximum:  63230
    - Standard Deviation:  991.128900299926

In [13]:
# Count all orders
all_orders = df['Net Orders'].sum()

# Count canceled orders
canceled_orders = df['Net Orders'].where(df['Net Orders'] < 0).sum()

# Count successful orders
successful_orders = df['Net Orders'].where(df['Net Orders'] > 0).sum()

print('All orders:', all_orders)
print('Canceled orders:', canceled_orders)
print('Successful orders:', successful_orders)



All orders: 3533679
Canceled orders: -5167.0
Successful orders: 3538846.0


In [14]:
# data2
univarite(df2)

Column:  Rpt Week
Unique Values:  15
Value Counts: 
 2018-W46    21
2018-W48    20
2018-W47    20
2019-W47    19
2018-W45    19
2019-W48    19
2019-W45    17
2019-W46    17
2019~W46     4
2019~W45     4
2018~W45     2
2019~W47     2
2019~W48     2
2018~W48     1
2018~W47     1
Name: Rpt Week, dtype: int64


Column:  Super Region
Unique Values:  3
Value Counts: 
 APAC     72
EMEA     48
LATAM    24
Name: Super Region, dtype: int64


Column:  Country Name
Unique Values:  7
Value Counts: 
 Norway            24
Australia         24
US                24
South Korea       24
United Kingdom    24
Hong Kong         24
Brazil            24
Name: Country Name, dtype: int64


Column:  Mobile Indicator Name
Unique Values:  5
Value Counts: 
 Mobile     91
Desktop    45
MOBILE     13
DESKTOP    11
MBL         8
Name: Mobile Indicator Name, dtype: int64


Column:  Platform Type Name
Unique Values:  3
Value Counts: 
 Mobile Web    56
Desktop       56
Mobile App    56
Name: Platform Type Name, dtype: i

Observation 
- Column : Rpt week
    - column requires cleaning ~ to - 

- Column : Super Region 
    - 3 main regions 
    - highest count from APAC

- Column : Country Name (Sending country)
    - count of 7 unique 
    - all with equal counts of 24 

- Column: Mobile Indicator Name 
    - Need cleaning, covert all to lower case

- Column: Platform Type Name
    - 3 unique types
    - no cleaning needed
    
- Column:  Visits
    - Mean:  437469.9285714286
    - Median:  155106.5
    - Minimum:  19654
    - Maximum:  3318809
    - Standard Deviation:  726875.0904956426


### Data Cleaning and Feature Engineering
- Now that we have taken a look at our data column by column . We can clean it up then do a final Univarite check.
- If everything is ok then we proceed to perform Feature Engineering.

In [15]:
#data1 cleaning
df['Week'] = df['Week'].replace('~', '-', regex=True)
df['Transfer Type'] = df['Transfer Type'].replace('~', '-', regex=True)
#df = df.applymap(lambda x: str(x).replace('~', '-')) #to replace ~ to - in both week column and transfer type
df['Mobile Indicator Name'].replace(['mobi','M@#3le', 'mibile'], 'Mobile', inplace=True) # change the misspells to Mobile in Mobile Indicator
df['Receive Country'] = df['Receive Country'].str.lower() # make all receiving country name to lowercase

#data2 cleaning
df2['Rpt Week'] = df2['Rpt Week'].replace('~', '-', regex=True)
df2['Mobile Indicator Name'] = df2['Mobile Indicator Name'].str.lower()

In [16]:
# during data understanding we found out that Super region had alot of null values lets find out why
df[df["Super Region"].isna()]['Country Name'].unique()


array(['US'], dtype=object)

In [17]:
# we observe that all the null super region are under US country name . According to google US is under North America . also our data defination say 4 regions (North America, Asia-Pacific, Europe Middle East & Africa or Latin America)
# from our univariate analyis we only see 3 
# lets check if there are any US Country names with Region column Not null , if none is present we will imput the initial NAM to reprsent North America

df_selected = df[(df['Super Region'].notna()) & (df['Country Name'] == 'US')]

In [18]:
df_selected

,Week,Mobile Indicator Name,Platform Type Name,Super Region,Country Name,Transfer Type,Receive Country,Net Sent Amount GBP,Net Orders


In [19]:
#same applies to Data 2
df2[df2["Super Region"].isna()]['Country Name'].unique()

array(['US'], dtype=object)

In [20]:
# all US country has no region attached to it 
df["Super Region"] = df["Super Region"].fillna("NAM")
df2["Super Region"] = df2["Super Region"].fillna("NAM")


unkown transactions and nan values in country check

Feature Engineering
- First Split the weeks into year and week number

In [21]:
df[['Year', 'Week Number']] = df['Week'].str.split("-", expand=True)
df2[['Year', 'Week Number']] = df2['Rpt Week'].str.split("-", expand=True)

Split Data between 2018 and 2019

In [22]:
df_2018 = df.query("Year == '2018'")
df_2019 = df.query("Year == '2019'")
df2_2018 = df2.query("Year == '2018'")
df2_2019 = df2.query("Year == '2019'")

## Bivariate Analysis

In [26]:
def compare(column_name1,column_name2):
    data_2018 = df_2018.groupby([column_name1, column_name2]).size().reset_index(name='counts_2018')
    data_2019 = df_2019.groupby([column_name1, column_name2]).size().reset_index(name='counts_2019')
    data_comparision = pd.merge(data_2018,data_2019, on=[column_name1, column_name2], how='outer')
    data_comparision['growth'] = round((data_comparision['counts_2019'] - data_comparision['counts_2018']) / data_comparision['counts_2018'] * 100)
    return data_comparision


We Are told that there is a significant improvement in mobile app usage, how big is the change?

In [23]:
platform_type_counts_2018 = df_2018['Platform Type Name'].value_counts()
platform_type_counts_2019 = df_2019['Platform Type Name'].value_counts()
percentage_change = round(((platform_type_counts_2019 - platform_type_counts_2018) / platform_type_counts_2018) * 100)

df_percentage_change = pd.concat([platform_type_counts_2018,platform_type_counts_2019, percentage_change], axis=1)
df_percentage_change.columns = ['2018', '2019', '% Change']
df_percentage_change

,2018,2019,% Change
Desktop,16726,18322,10.0
Mobile Web,9092,10760,18.0
Mobile App,7872,10916,39.0


In [49]:
plot_line(platform_type_counts_2018,platform_type_counts_2019,"Platform Type Name","Count")

How many Orders where made through Mobile App and Whats the Growth Like?

In [48]:
order_2018 = df_2018.groupby(['Platform Type Name']).aggregate({'Net Orders': 'sum'})
order_2019 = df_2019.groupby(['Platform Type Name']).aggregate({'Net Orders': 'sum'})
order_2018 = order_2018.rename(columns={'Net Orders': 'Net Orders_2018'})
order_2019 = order_2019.rename(columns={'Net Orders': 'Net Orders_2019'})

df_merged = pd.merge(order_2018, order_2019, on='Platform Type Name', suffixes=('_2018', '_2019'))
df_merged['growth'] = round((df_merged['Net Orders_2019']-df_merged['Net Orders_2018'])/df_merged['Net Orders_2018']*100)
df_merged

,Net Orders_2018,Net Orders_2019,growth
Platform Type Name,,,
Desktop,1066185,1211274,14.0
Mobile App,183998,332541,81.0
Mobile Web,286114,453567,59.0


In [50]:
plot_line(order_2018,order_2019,'Platform Type Name','Net Order')

Larger picture : what are the platform counts per region per year


In [62]:
platform_region =  compare('Super Region', 'Platform Type Name')
platform_region

,Super Region,Platform Type Name,counts_2018,counts_2019,growth
0,APAC,Desktop,5281,5773,9.0
1,APAC,Mobile App,2807,3902,39.0
2,APAC,Mobile Web,2781,3261,17.0
3,EMEA,Desktop,5447,6038,11.0
4,EMEA,Mobile App,2206,3092,40.0
5,EMEA,Mobile Web,3038,3656,20.0
6,LATAM,Desktop,1558,1759,13.0
7,LATAM,Mobile App,471,839,78.0
8,LATAM,Mobile Web,693,821,18.0
9,NAM,Desktop,4440,4752,7.0


In [30]:
create_stacked_bar(platform_region,'Super Region', "Platform Type Name")

Observation : In all the 4 regions Mobile Apps  platform types has had the highest growth . LATAM leading with 78%. Combining Mobile App and Mobile web gives us LATAM (Brazil) with Highest growth.

Question : Which countriees are using Mobile Apps to make orders and how is the growth 2018 vs 2019

In [51]:
platform_country =  compare("Receive Country", "Platform Type Name")
platform_country =   platform_country[platform_country["Platform Type Name"].isin(['Mobile App'])].sort_values('counts_2019',ascending=False)
platform_country

,Receive Country,Platform Type Name,counts_2018,counts_2019,growth
342,nigeria,Mobile App,262.0,282.0,8.0
183,ghana,Mobile App,225.0,261.0,16.0
249,kenya,Mobile App,219.0,252.0,15.0
474,thailand,Mobile App,223.0,241.0,8.0
235,italy,Mobile App,208.0,232.0,12.0
...,...,...,...,...,...
7,algria,Mobile App,1.0,NaN,NaN
10,andora,Mobile App,1.0,NaN,NaN
17,anguilla,Mobile App,1.0,NaN,NaN
218,ind,Mobile App,1.0,NaN,NaN


In [52]:
create_stacked_bar(platform_country[:3],'Receive Country', "Platform Type Name")

which sending country to receiving country  combination had the highest mobile money transfer


In [61]:

highest_transfer_2018 = df_2018[df_2018['Platform Type Name'] == 'Mobile App'].groupby(['Country Name', 'Receive Country'])['Net Orders'].sum().sort_values(ascending=False).reset_index()
highest_transfer_2019 = df_2019[df_2019['Platform Type Name'] == 'Mobile App'].groupby(['Country Name', 'Receive Country'])['Net Orders'].sum().sort_values(ascending=False).reset_index()
highest_transfer_2019

,Country Name,Receive Country,Net Orders
0,US,nigeria,209996
1,United Kingdom,ghana,15075
2,South Korea,pakistan,9926
3,South Korea,japan,5731
4,Australia,kenya,4481
...,...,...,...
758,Australia,monaco,-1
759,South Korea,uzbekistan,-1
760,Australia,jordan,-1
761,United Kingdom,brunei,-2


Which countries are ordering from brazil

which Country Made more out of mobile app platform


In [63]:
df_2018_mobile = df_2018[df_2018['Platform Type Name'] == 'Mobile App'].groupby('Country Name').sum().sort_values('Net Sent Amount GBP', ascending=False)
df_2019_mobile = df_2019[df_2019['Platform Type Name'] == 'Mobile App'].groupby('Country Name').sum().sort_values('Net Sent Amount GBP', ascending=False)
df_compare = df_2018_mobile.join(df_2019_mobile, rsuffix='_2019')
df_compare = df_compare.rename(columns={'Net Sent Amount GBP':'Net Sent Amount GBP_2018'})

# calculate the growth in percentage
df_compare['growth'] = round((df_compare['Net Sent Amount GBP_2019']-df_compare['Net Sent Amount GBP_2018'])/df_compare['Net Sent Amount GBP_2018']*100)
df_compare

/var/folders/7f/86x8mq4s30g32jbtrqbx058c0000gn/T/ipykernel_3940/3373255456.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_2018_mobile = df_2018[df_2018['Platform Type Name'] == 'Mobile App'].groupby('Country Name').sum().sort_values('Net Sent Amount GBP', ascending=False)
/var/folders/7f/86x8mq4s30g32jbtrqbx058c0000gn/T/ipykernel_3940/3373255456.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_2019_mobile = df_2019[df_2019['Platform Type Name'] == 'Mobile App'].groupby('Country Name').sum().sort_values('Net Sent Amount GBP', ascending=False)


,Net Sent Amount GBP_2018,Net Orders,Net Sent Amount GBP_2019,Net Orders_2019,growth
Country Name,,,,,
US,2.645716e+07,145149,4.501073e+07,238810,70.0
United Kingdom,3.159200e+06,13421,5.446722e+06,24757,72.0
South Korea,2.417928e+06,10747,7.858534e+06,35498,225.0
Hong Kong,1.384523e+06,5459,3.280680e+06,14326,137.0
Australia,1.137545e+06,3758,5.690167e+07,7418,4902.0
Norway,9.702930e+05,3084,1.395330e+06,5273,44.0
Brazil,6.717008e+05,2380,1.552326e+06,6459,131.0


In [64]:
bar = Bar()
bar.add_xaxis(df_compare.index.tolist())
bar.add_yaxis("2018", df_compare['Net Sent Amount GBP_2018'].tolist(), stack='stack')
bar.add_yaxis("2019", df_compare['Net Sent Amount GBP_2019'].tolist(), stack='stack')
bar.set_global_opts(xaxis_opts={"name":"Country Name"}, yaxis_opts={"name":"Net Sent Amount GBP"})
bar.render_notebook()

Which week made most orders through app

In [68]:
df_2018_mobile_app = df_2018[df_2018['Platform Type Name'] == 'Mobile App']
df_2018_week_app = df_2018_mobile_app.groupby(['Week Number'])['Net Orders'].sum().reset_index()

df_2019_mobile_app = df_2019[df_2019['Platform Type Name'] == 'Mobile App']
df_2019_week_app = df_2019_mobile_app.groupby(['Week Number'])['Net Orders'].sum().reset_index()

df_week_app = pd.merge(df_2018_week_app, df_2019_week_app, on='Week Number', suffixes=('_2018', '_2019'))
df_week_app['growth_%'] = ((df_week_app['Net Orders_2019'] - df_week_app['Net Orders_2018'])/df_week_app['Net Orders_2018'])*100

In [76]:
df_week_app

,Week Number,Net Orders_2018,Net Orders_2019,growth_%
0,W45,46740,89044,90.509200
1,W46,46593,81283,74.453244
2,W47,44696,83028,85.761589
3,W48,45952,79186,72.323294


In [75]:
line = Line()
line.add_xaxis(df_week_app['Week Number'].tolist())
line.add_yaxis("2018", df_week_app['Net Orders_2018'].tolist(), is_smooth=True,label_opts={"position": "top"})
line.add_yaxis("2019", df_week_app['Net Orders_2019'].tolist(), is_smooth=True,label_opts={"position": "top"})
line.set_global_opts(title_opts={"text":"Net Orders by Week (Mobile App)"}, xaxis_opts={"name":"Week Number"}, yaxis_opts={"name":"Net Orders"})
line.render_notebook()

In [87]:
#week performance and country

df_week_rc_2018 = df_2018[df_2018["Platform Type Name"] == "Mobile App"].groupby(["Week Number", "Receive Country"])["Net Orders"].sum().reset_index()


df_week_rc_2019 = df_2019[df_2019["Platform Type Name"] == "Mobile App"].groupby(["Week Number", "Receive Country"])["Net Orders"].sum().reset_index()

# merge the two dataframes on week and receive country
df_week_rc = pd.merge(df_week_rc_2018, df_week_rc_2019, on=["Week Number", "Receive Country"], suffixes=("_2018", "_2019"))


df_week_rc['growth_pct'] = ((df_week_rc['Net Orders_2019'] - df_week_rc['Net Orders_2018']) / df_week_rc['Net Orders_2018']) * 100
df_week_rc = df_week_rc.sort_values(by='growth_pct', ascending=False)

top3_per_week = df_week_rc.groupby('Week Number').apply(lambda x: x.nlargest(3,'Net Orders_2019'))
top3_per_week

Week Number Receive Country  Net Orders_2018  Net Orders_2019  \
Week Number                                                                     
W45         83          W45         nigeria            34517            59716   
            42          W45           ghana             2339             4757   
            86          W45        pakistan              909             2375   
W46         213         W46         nigeria            34002            50835   
            171         W46           ghana             2493             4312   
            216         W46        pakistan             1008             3143   
W47         346         W47         nigeria            32734            54279   
            302         W47           ghana             2296             4112   
            349         W47        pakistan              937             2987   
W48         478         W48         nigeria            32606            51225   
            437         W48           ghana             2566             4111   
            481         W48        pakistan             1112             2845   

                 growth_pct  
Week Number                  
W45         83    73.004606  
            42   103.377512  
            86   161.276128  
W46         213   49.505911  
            171   72.964300  
            216  211.805556  
W47         346   65.818415  
            302   79.094077  
            349  218.783351  
W48         478   57.102987  
            437   60.210444  
            481  155.845324

WHICH WEEK MADE MOST MONEY THOUGH APP

In [73]:
df_2018_mobile_money = df_2018[df_2018['Platform Type Name'] == 'Mobile App']
df_2018_week_amount = df_2018_mobile_money.groupby(['Week Number'])['Net Sent Amount GBP'].sum().reset_index()

df_2019_mobile_money = df_2019[df_2019['Platform Type Name'] == 'Mobile App']
df_2019_week_amount = df_2019_mobile_money.groupby(['Week Number'])['Net Sent Amount GBP'].sum().reset_index()

df_week_amount = pd.merge(df_2018_week_amount, df_2019_week_amount, on='Week Number', suffixes=('_2018', '_2019'))
df_week_amount['growth_%'] = ((df_week_amount['Net Sent Amount GBP_2019'] - df_week_amount['Net Sent Amount GBP_2018'])/df_week_amount['Net Sent Amount GBP_2018'])*100

In [85]:
df_week_amount

,Week Number,Net Sent Amount GBP_2018,Net Sent Amount GBP_2019,growth_%
0,W45,9.642783e+06,1.895385e+07,96.559942
1,W46,9.411764e+06,1.665724e+07,76.983151
2,W47,8.354428e+06,1.579814e+07,89.098989
3,W48,8.781813e+06,7.003676e+07,697.520571


In [74]:
line = Line()
line.add_xaxis(df_week_amount['Week Number'].tolist())
line.add_yaxis("2018", df_week_amount['Net Sent Amount GBP_2018'].tolist(), is_smooth=True,label_opts={"position": "top"})
line.add_yaxis("2019", df_week_amount['Net Sent Amount GBP_2019'].tolist(), is_smooth=True,label_opts={"position": "top"})
line.set_global_opts(title_opts={"text":"Net Amount GBP by Week (Mobile Money)"}, xaxis_opts={"name":"Week Number"}, yaxis_opts={"name":"Net Sent Amount GBP"})
line.render_notebook()


In [89]:
df2_2018

,Rpt Week,Super Region,Country Name,Mobile Indicator Name,Platform Type Name,Visits,Year,Week Number
2,2018-W48,NAM,US,desktop,Desktop,2492783,2018,W48
10,2018-W45,APAC,Australia,desktop,Desktop,166223,2018,W45
13,2018-W47,EMEA,United Kingdom,mobile,Mobile App,107998,2018,W47
17,2018-W47,APAC,Australia,mobile,Mobile App,32212,2018,W47
18,2018-W48,NAM,US,mobile,Mobile Web,1807551,2018,W48
...,...,...,...,...,...,...,...,...
161,2018-W47,LATAM,Brazil,desktop,Desktop,312746,2018,W47
162,2018-W46,APAC,Australia,mobile,Mobile App,33145,2018,W46
165,2018-W48,LATAM,Brazil,desktop,Desktop,297762,2018,W48
166,2018-W48,EMEA,United Kingdom,mobile,Mobile App,108467,2018,W48


In [92]:
visits_2018 = df2_2018.groupby(['Platform Type Name']).aggregate({'Visits': 'sum'})
visits_2019 = df2_2019.groupby(['Platform Type Name']).aggregate({'Visits': 'sum'})
visits_2018 = visits_2018.rename(columns={'Visits': 'Visits_2018'})
visits_2019 = visits_2019.rename(columns={'Visits': 'Visits_2019'})


df_merged = pd.merge(visits_2018, visits_2019, on='Platform Type Name', suffixes=('_2018', '_2019'))
df_merged['growth'] = round((df_merged['Visits_2019']-df_merged['Visits_2018'])/df_merged['Visits_2018']*100)
df_merged

,Visits_2018,Visits_2019,growth
Platform Type Name,,,
Desktop,17124508,20254316,18.0
Mobile App,4118014,6380492,55.0
Mobile Web,11201598,14416020,29.0


In [101]:
plot_line(visits_2018,visits_2019,'Platform Type Name','Visits')

In [100]:
df2_2018_mobile_visits = df2_2018[df2_2018['Platform Type Name'] == 'Mobile App']
df2_2018_week_visits = df2_2018_mobile_visits.groupby(['Week Number'])['Visits'].sum().reset_index()

df2_2019_mobile_visits = df2_2019[df2_2019['Platform Type Name'] == 'Mobile App']
df2_2019_week_visits = df2_2019_mobile_visits.groupby(['Week Number'])['Visits'].sum().reset_index()

df2_week_app = pd.merge(df2_2018_week_visits, df2_2019_week_visits, on='Week Number', suffixes=('_2018', '_2019'))
df2_week_app['growth_%'] = ((df2_week_app['Visits_2019'] - df2_week_app['Visits_2018'])/df2_week_app['Visits_2018'])*100
df2_week_app

,Week Number,Visits_2018,Visits_2019,growth_%
0,W45,1054657,1634713,54.999493
1,W46,1046428,1592966,52.228916
2,W47,1013263,1584363,56.362465
3,W48,1003666,1568450,56.272106


In [99]:
line = Line()
line.add_xaxis(df2_week_app['Week Number'].tolist())
line.add_yaxis("2018", df2_week_app['Visits_2018'].tolist(), is_smooth=True,label_opts={"position": "top"})
line.add_yaxis("2019", df2_week_app['Visits_2019'].tolist(), is_smooth=True,label_opts={"position": "top"})
line.set_global_opts(title_opts={"text":"Visits by Week (Mobile App)"}, xaxis_opts={"name":"Week Number"}, yaxis_opts={"name":"visits"})
line.render_notebook()

Conversion of Visits to orders and to Sales

## EDA for other columns None Mobile app platform type name

In [54]:
transfer_type_counts_2018 = df_2018['Transfer Type'].value_counts()
transfer_type_counts_2019 = df_2019['Transfer Type'].value_counts()
percentage_change = round(((transfer_type_counts_2019 - transfer_type_counts_2018) / transfer_type_counts_2018) * 100)

df_percentage_change = pd.concat([transfer_type_counts_2018,transfer_type_counts_2019, percentage_change], axis=1)
df_percentage_change.columns = ['2018', '2019', '% Change']
df_percentage_change
#rank per highest to lowest

,2018,2019,% Change
Same Day Cash Pick Up,4282,5049,18.0
3-7 Day Cash Pick Up,4136,4944,20.0
Mobile Money,3901,4628,19.0
Next Day Bank Transfer,3792,4451,17.0
Next Day Cash Pick Up,3757,4418,18.0
3-7 Day Bank Transfer,3733,4351,17.0
Same Day Bank Transfer,3653,4365,19.0
Slow Bank Transfer,3263,3868,19.0
Slow Cash Pick Up,3161,3896,23.0
Unknown,12,28,133.0


In [55]:
plot_line(transfer_type_counts_2018,transfer_type_counts_2019,"Transfer Type","Count")

Obervation : 
- Also there is a Growth of Unkown type transactions, Are they a result of some sort of fraud? also which receiving country has most Unkown type?

From what region/country are we getting the unkown transfer type?

In [40]:


# Next, filter out rows where 'Transfer Type' is 'Unknown'
df_2018_unknown = df_2018[df_2018['Transfer Type'] == 'Unknown']
df_2019_unknown = df_2019[df_2019['Transfer Type'] == 'Unknown']

# Finally, groupby 'Super Region' or 'Country Name' and get the count of rows for each group
unknown_by_region_2018 = df_2018_unknown.groupby('Super Region').size().reset_index(name='counts_2018')
unknown_by_region_2019 = df_2019_unknown.groupby('Super Region').size().reset_index(name='counts_2019')

# or
unknown_by_country_2018 = df_2018_unknown.groupby('Country Name').size().reset_index(name='counts_2018')
unknown_by_country_2019 = df_2019_unknown.groupby('Country Name').size().reset_index(name='counts_2019')


In [41]:
unknown_by_country_2019

,Country Name,counts_2019
0,Australia,1
1,Norway,2
2,South Korea,1
3,US,22
4,United Kingdom,2


What platform type have the unkown transfer type to them?

Transfer type "Mobile money"  per region 

In [42]:
mobile_transfer_region =  compare('Super Region', 'Transfer Type')

In [43]:
mobile_transfer_region =  compare('Super Region', 'Transfer Type')
mob = mobile_transfer_region[mobile_transfer_region["Transfer Type"].isin(['Mobile Money'])]

In [44]:
mob

,Super Region,Transfer Type,counts_2018,counts_2019,growth
2,APAC,Mobile Money,1293.0,1515,17.0
11,EMEA,Mobile Money,1252.0,1497,20.0
20,LATAM,Mobile Money,377.0,480,27.0
29,NAM,Mobile Money,979.0,1136,16.0


In [45]:
mob = mob.groupby(["Super Region"]).sum()
mob = mob.rename(columns={"counts_2018": "2018", "counts_2019": "2019"})
mob = mob[['2018','2019']]
pie = Pie()
pie.add("2018", [list(z) for z in zip(mob.index.tolist(), mob["2018"].tolist())],center=["25%", "50%"])
pie.add("2019", [list(z) for z in zip(mob.index.tolist(), mob["2019"].tolist())],center=["70%", "50%"])
pie.set_global_opts(title_opts={"text":"Mobile Money Transfer Type by Region"})
pie.render_notebook()

/var/folders/7f/86x8mq4s30g32jbtrqbx058c0000gn/T/ipykernel_3620/2063115809.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  mob = mob.groupby(["Super Region"]).sum()


whats the growth per country between 2018 and 19

In [46]:
mobile_transfer_country =  compare('Country Name', 'Transfer Type')
mobile_transfer_country[mobile_transfer_country["Transfer Type"].isin(['Mobile Money'])]

,Country Name,Transfer Type,counts_2018,counts_2019,growth
2,Australia,Mobile Money,513.0,553,8.0
11,Brazil,Mobile Money,377.0,480,27.0
20,Hong Kong,Mobile Money,361.0,438,21.0
29,Norway,Mobile Money,501.0,560,12.0
38,South Korea,Mobile Money,419.0,524,25.0
47,US,Mobile Money,979.0,1136,16.0
57,United Kingdom,Mobile Money,751.0,937,25.0


In [47]:
create_stacked_bar(mobile_transfer_country,'Country Name', 'Transfer Type')

are only mobile user using mobile money

In [50]:
mobile_transfer_platform =  compare('Platform Type Name','Transfer Type')

In [51]:
mobile_transfer_platform = mobile_transfer_platform[mobile_transfer_platform["Transfer Type"].isin(['Mobile Money'])]

In [52]:
mobile_transfer_platform

,Platform Type Name,Transfer Type,counts_2018,counts_2019,growth
2,Desktop,Mobile Money,1992,2178,9.0
12,Mobile App,Mobile Money,789,1163,47.0
22,Mobile Web,Mobile Money,1120,1287,15.0


In [43]:
create_stacked_bar(mobile_transfer_platform,'Platform Type Name','Transfer Type')

which country made most money from mobile money transfers

In [44]:
df_2018_mobile_money = df_2018[df_2018['Transfer Type'] == 'Mobile Money'].groupby('Country Name').sum().sort_values('Net Sent Amount GBP', ascending=False)
df_2019_mobile_money = df_2019[df_2019['Transfer Type'] == 'Mobile Money'].groupby('Country Name').sum().sort_values('Net Sent Amount GBP', ascending=False)
df_compare = df_2018_mobile_money.join(df_2019_mobile_money, rsuffix='_2019')
df_compare = df_compare.rename(columns={'Net Sent Amount GBP':'Net Sent Amount GBP_2018'})

# calculate the growth in percentage
df_compare['growth'] = round((df_compare['Net Sent Amount GBP_2019']-df_compare['Net Sent Amount GBP_2018'])/df_compare['Net Sent Amount GBP_2018']*100)

/var/folders/7f/86x8mq4s30g32jbtrqbx058c0000gn/T/ipykernel_3074/3393245221.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_2018_mobile_money = df_2018[df_2018['Transfer Type'] == 'Mobile Money'].groupby('Country Name').sum().sort_values('Net Sent Amount GBP', ascending=False)
/var/folders/7f/86x8mq4s30g32jbtrqbx058c0000gn/T/ipykernel_3074/3393245221.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_2019_mobile_money = df_2019[df_2019['Transfer Type'] == 'Mobile Money'].groupby('Country Name').sum().sort_values('Net Sent Amount GBP', ascending=False)


In [45]:
df_compare

,Net Sent Amount GBP_2018,Net Orders,Net Sent Amount GBP_2019,Net Orders_2019,growth
Country Name,,,,,
US,2.072623e+07,33039,2.385113e+07,40463,15.0
United Kingdom,6.773866e+06,12732,9.484025e+06,18631,40.0
Norway,3.883277e+06,4724,3.964500e+06,5710,2.0
Australia,2.868950e+06,5007,6.772312e+07,4799,2261.0
Brazil,2.367592e+06,3118,2.076400e+06,4237,-12.0
Hong Kong,1.526594e+06,3256,1.714165e+06,4176,12.0
South Korea,8.849491e+05,1989,1.618880e+06,4115,83.0


In [46]:
bar = Bar()
bar.add_xaxis(df_compare.index.tolist())
bar.add_yaxis("2018", df_compare['Net Sent Amount GBP_2018'].tolist(), stack='stack')
bar.add_yaxis("2019", df_compare['Net Sent Amount GBP_2019'].tolist(), stack='stack')
bar.set_global_opts(xaxis_opts={"name":"Country Name"}, yaxis_opts={"name":"Net Sent Amount GBP"})
bar.render_notebook()


lets check the relationship between Country Name and number of orders and amount made through mobile money ########check on this

In [50]:
# Group the data by 'Country Name' and 'Year'
df_2018_grouped = df_2018.groupby(['Country Name', 'Year']).agg({'Net Orders': 'sum', 'Net Sent Amount GBP': 'sum'}).reset_index()
df_2019_grouped = df_2019.groupby(['Country Name', 'Year']).agg({'Net Orders': 'sum', 'Net Sent Amount GBP': 'sum'}).reset_index()

# Merge the data for 2018 and 2019
df_merged = pd.merge(df_2018_grouped, df_2019_grouped, on='Country Name', suffixes=('_2018', '_2019'))

# Calculate the percentage growth between 2018 and 2019
df_merged['Orders_Growth'] = ((df_merged['Net Orders_2019'] - df_merged['Net Orders_2018']) / df_merged['Net Orders_2018'])*100
df_merged['Amount_Growth'] = ((df_merged['Net Sent Amount GBP_2019'] - df_merged['Net Sent Amount GBP_2018']) / df_merged['Net Sent Amount GBP_2018'])



In [82]:
df_merged

,Country Name,Year_2018,Net Orders_2018,Net Sent Amount GBP_2018,Year_2019,Net Orders_2019,Net Sent Amount GBP_2019,Orders_Growth,Amount_Growth
0,Australia,2018,38891,1.464080e+07,2019,49208,3.980989e+08,26.527988,26.191071
1,Brazil,2018,36665,1.650281e+07,2019,55954,1.631560e+07,52.608755,-0.011345
2,Hong Kong,2018,33714,1.120491e+07,2019,52518,1.508776e+07,55.775049,0.346531
3,Norway,2018,40984,1.750574e+07,2019,48621,1.646838e+07,18.634101,-0.059258
4,South Korea,2018,55350,1.611105e+07,2019,104122,2.785014e+07,88.115628,0.728636
5,US,2018,1182588,2.956377e+08,2019,1475812,3.564179e+08,24.795110,0.205590
6,United Kingdom,2018,147836,4.531458e+07,2019,211009,6.016508e+07,42.731811,0.327720
